## Prepare your dataset

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import tqdm

def load_mnist_data(root_path='./data', batch_size=4):
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))]
    )

    trainset = torchvision.datasets.MNIST(root=root_path, train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    testset = torchvision.datasets.MNIST(root=root_path, train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    return trainloader, testloader

## Building your neural network

In [2]:
import numpy as np
from typing import Any, Callable, Tuple

##################################
# For matrices or arbitrary size #
##################################
class MyWeightTensor:
    def __init__(self, shape: Tuple or int, init_weight_fn: Callable = np.random.randn, init_weights: 'MyWeightTensor' or np.ndarray or int or float = None):
        assert isinstance(shape, tuple) or isinstance(shape, int) or isinstance(shape, float), f'Allowed shapes: tuple, int, float, got: {type(shape)}'
        self.shape = shape

        if init_weights is not None:
            if isinstance(init_weights, MyWeightTensor):
                self.values = init_weights.values
            else:
                if isinstance(shape, tuple):
                    assert isinstance(init_weights, np.ndarray)
                else:
                    assert isinstance(init_weights, int) or isinstance(init_weights, float)
                
                self.values = init_weights
        else:
            if isinstance(shape, int):
                self.shape = (self.shape,)
                self.values = init_weight_fn(shape)
            else:
                self.values = init_weight_fn(*shape)
    
    @property
    def T(self) -> 'MyWeightTensor':
        _T = self.values.T
        return MyWeightTensor(shape=_T.shape, init_weights=_T)
    
    def __add__(self, other) -> 'MyWeightTensor':
        if isinstance(other, MyWeightTensor):
            other = other.values
        else:
            assert isinstance(other, np.ndarray) or isinstance(other, int) or isinstance(other, float)
        
        return MyWeightTensor(shape=self.values.shape, init_weights=self.values + other)

    def __mul__(self, other) -> 'MyWeightTensor':
        if isinstance(other, MyWeightTensor):
            other = other.values
        else:
            assert isinstance(other, np.ndarray) or isinstance(other, int) or isinstance(other, float)
        
        _dot = np.dot(self.values, other)

        return MyWeightTensor(shape=_dot.shape, init_weights=_dot)


###############################
# For creating a linear layer #
###############################
class MyLinearLayer:
    def __init__(self, in_features: int, out_features: int, init_weight_fn: Callable = np.random.randn) -> None:
        self.in_features = in_features
        self.out_features = out_features

        self.weights = MyWeightTensor(shape=(out_features, in_features), init_weight_fn=init_weight_fn)
        self.bias = MyWeightTensor(shape=out_features, init_weight_fn=init_weight_fn)

        self.latest_input = None
        self.latest_output = None

    def __call__(self, tensor: np.ndarray or MyWeightTensor) -> MyWeightTensor:
        self.latest_input = tensor

        bs = -1
        if len(tensor.shape) == 2:
            # batch size included
            bs = tensor.shape[0]
            _w = self.weights * tensor.T
        else:
            _w = self.weights * tensor
        
        _bias = self.bias.values
        if bs != -1:
            _bias = np.tile(_bias, bs).reshape(bs, -1)
        
        self.latest_output = (_w + _bias.T).T

        return MyWeightTensor(shape=self.latest_output.shape, init_weights=self.latest_output)
    
    def derivative(self) -> float:
        assert self.latest_output is not None, 'Cannot calculate grad without a single forward pass.'
        # Linear activation derivation
        return np.ones(shape=self.latest_output.shape)

In [3]:
####################################
# Creating a custom neural network #
####################################

def xavier_normal_init(*shape) -> np.ndarray:
    assert len(shape) <= 2, 'Can only init max 2d tensors'
    fan_in = shape[0]
    if len(shape) == 1:
        fan_out = fan_in
    else:
        fan_out = shape[1]
    gain = 1.0

    std = gain * np.sqrt(2.0 / (fan_in + fan_out))
    return np.random.normal(loc=0.0, scale=std, size=shape)


class MyNeuralNetwork:
    def __init__(self) -> None:
        # init_weight_fn = lambda *shape: np.random.randn(*shape) / 10
        init_weight_fn = lambda *shape: xavier_normal_init(*shape)
        self.layers = [
            MyLinearLayer(in_features=784, out_features=32, init_weight_fn=init_weight_fn),
            MyLinearLayer(in_features=32, out_features=32, init_weight_fn=init_weight_fn),
            MyLinearLayer(in_features=32, out_features=10, init_weight_fn=init_weight_fn)
        ]
    
    def __call__(self, tensor: np.ndarray) -> Any:
        x = tensor
        for layer in self.layers:
            x = layer(x)
        
        return x

## Implement your loss function

In [4]:
import torch.nn.functional as F


def softmax(input: np.ndarray) -> np.ndarray:
    _softmax = np.asarray([np.exp(_in) /np.sum(np.exp(_in), axis=0) for _in in input])

    return _softmax


class CrossEntropyLoss:
    def __init__(self) -> None:
        pass

    def __call__(self, predictions: MyWeightTensor or np.ndarray, targets: MyWeightTensor or np.ndarray) -> np.ndarray:
        """
        Computes cross entropy between targets snd predictions.    
        Returns: List of cross entropy losses (batch-wise)
        """
        if isinstance(predictions, MyWeightTensor):
            predictions = predictions.values
        
        if isinstance(targets, MyWeightTensor):
            targets = targets.values

        assert predictions.shape[0] == targets.shape[0]
        if len(targets.shape) == 2:
            targets = targets.reshape(-1)
        predictions = torch.as_tensor(predictions)
        targets = torch.as_tensor(targets)

        loss = np.array([F.cross_entropy(pred, t).item() for pred, t in zip(predictions, targets)])

        return loss
    
    def derivative(self) -> Callable:
        # y_hat is the prediction
        # y is the target value
        def _derivative(y_hat: MyWeightTensor or np.ndarray, y: MyWeightTensor or np.ndarray) -> np.ndarray:
            if isinstance(y_hat, MyWeightTensor):
                y_hat = y_hat.values
            
            if isinstance(y, MyWeightTensor):
                y = y.values

            _y = np.zeros(shape=y_hat.shape)
            np.put_along_axis(_y, y, 1, axis=-1)

            y_hat = softmax(y_hat)

            return y_hat - _y
        
        return _derivative

## Implement the training loop

In [5]:
def train(model: MyNeuralNetwork, batch_size: int, learning_rate: float, loss_fn: Callable, epochs: int = 10):
    train_loader, _ = load_mnist_data(batch_size=batch_size)

    for epoch in range(epochs):
        running_loss = 0.0
        running_accuracy = []
        for imgs, targets in tqdm.tqdm(train_loader, desc=f'Training iteration {epoch + 1}'):

            # for custom model
            imgs = imgs.numpy()
            targets = targets.numpy()

            if len(targets.shape) == 1:
                targets = targets.reshape(-1, 1)

            imgs = imgs.reshape(-1, 28 * 28)

            imgs = MyWeightTensor(shape=imgs.shape, init_weights=imgs)

            outputs = model(imgs).values

            loss = loss_fn(outputs, targets)
            print(loss)
            avg_loss = np.mean(loss)

            # print statistics
            running_loss += avg_loss

            # Calculate the Accuracy (how many of all samples are correctly classified?)
            max_outputs = np.argmax(outputs, axis=1)
            accuracy = (max_outputs == targets.flatten()).mean()
            running_accuracy.append(accuracy)

            #########################
            # Start backpropagation #
            #########################

            # Your code for backpropagation!


            #######################
            # End backpropagation #
            #######################

        print(f'Epoch {epoch + 1} finished with loss: {running_loss / len(train_loader):.3f} and accuracy: {torch.tensor(running_accuracy).mean():.3f}')

In [6]:
#############################
# Execute the training loop #
#############################
model = MyNeuralNetwork()
batch_size = 4
learning_rate = 0.001
epochs = 10
loss_fn = CrossEntropyLoss()

train(
    model=model,
    batch_size=batch_size,
    learning_rate=learning_rate,
    epochs=epochs,
    loss_fn=loss_fn
)

Training iteration 1:   1%|          | 107/15000 [00:00<00:37, 395.90it/s]

[2.21251621 2.62722012 4.08997173 4.16832853]
[3.75657256 0.74680998 5.84860971 9.24597745]
[8.95500462 5.16231383 3.50316857 4.13994585]
[4.44727052 9.21176537 5.53796637 4.56394185]
[3.89086537 2.32854035 0.99426957 7.06007515]
[0.33984084 8.95206481 2.05298541 3.54978461]
[0.12355097 2.79234607 2.34648956 3.27133829]
[4.21024965 5.49897468 5.10306077 5.92708084]
[5.05114888 3.92546176 2.74594077 7.17290702]
[3.21881353 2.37229205 3.29993042 3.42745593]
[0.85105006 1.5115467  9.71950615 4.0915773 ]
[4.74296926 3.9385491  4.71441908 4.18360347]
[7.20895865 4.75326218 2.66396456 4.85465451]
[8.93353747 3.64126646 6.07202875 4.19425316]
[2.28270079 9.95008363 5.60091488 4.05527051]
[5.01494175 3.68537247 4.93773897 4.38240209]
[4.5369737  6.04379049 3.41607592 3.98014592]
[1.69473025 5.95532904 3.8600434  2.30631495]
[2.08673323 4.43486478 9.15400642 3.24576811]
[6.70341074 4.52607969 5.98445369 7.05138687]
[3.97631878 3.1543811  5.00954738 3.15892566]
[2.92299136 4.66982648 5.17468497 

Training iteration 1:   1%|▏         | 218/15000 [00:00<00:30, 488.96it/s]

[2.26910432 4.07815604 2.563597   3.90034096]
[3.26947245 5.37272111 2.76111586 3.75586252]
[1.55698005 4.89375169 4.45107289 7.41604967]
[3.79804076 3.64155269 2.65599313 5.32627492]
[4.09316159 3.05420396 4.67171884 5.83847909]
[3.88010811 4.8709957  1.71572268 8.61132363]
[4.38492691 5.80100374 7.53653508 2.32972129]
[4.33599947 3.20366044 2.61545847 3.84427427]
[8.25351881 3.5541327  2.51731571 7.25655753]
[1.9032581  6.05507677 3.95437675 3.50238837]
[3.74277014 3.59863881 4.86661413 8.62556604]
[6.07263428 1.58301331 5.93911111 1.43221284]
[8.76192872 4.62529767 4.14396295 9.02668804]
[8.21846204 6.38440598 6.4394113  8.77006912]
[7.1520097  8.68653875 3.27211747 8.77738814]
[2.14952056 7.21071614 2.680151   4.08013269]
[1.28510974 2.34792663 0.93202935 3.79232315]
[4.71251804 4.70144274 0.89447577 5.94415825]
[3.57484359 5.34461987 2.99631321 0.23208882]
[4.12492184 3.88534578 4.83823093 3.12140147]
[5.44143333 4.2934927  3.61165706 3.87591066]
[3.73574595 5.4345527  5.65807546 

Training iteration 1:   2%|▏         | 268/15000 [00:00<00:38, 385.16it/s]

...

Training iteration 1:  23%|██▎       | 3510/15000 [00:06<00:20, 556.07it/s]

[4.41507253 2.42215754 5.01464674 7.13948591]
[7.06772113 4.47290738 2.68691923 3.58302841]
[7.80729421 7.14706478 1.98938894 0.16077207]
[3.94699416 5.45203953 3.37272716 1.35445834]
[8.2884849  5.54951475 0.49848065 5.40278463]
[4.01903423 5.56686438 3.94647171 6.33795902]
[1.35626469 7.13508379 3.44638422 9.14162605]
[3.5695903  2.6758116  2.86155763 9.17014014]
[1.9692067  8.9589238  3.27591018 4.53254502]
[8.87131249 6.3385215  3.82177339 1.73090873]
[4.39195626 5.726367   5.65253185 4.63498303]
[2.34194259 4.43341307 4.79639751 3.69735473]
[0.16272046 2.40258032 4.67943162 6.51166702]
[7.24641005 3.06253045 5.0624649  0.25747572]
[7.53470775 0.13475951 4.86385985 2.64922827]
[1.35565293 3.93278087 4.81905851 3.34535344]
[7.92194098 0.47965197 4.00798231 4.35547142]
[6.10187706 0.3524869  4.50575573 8.04978435]
[7.78200847 4.22547197 5.86611617 2.80495536]
[4.7950549  3.44342506 4.16154998 5.51479083]
[4.54279091 5.80448611 5.41184416 4.9221105 ]
[3.47856994 4.37695567 1.5488045  

Training iteration 1:  24%|██▍       | 3580/15000 [00:06<00:21, 533.80it/s]


[4.73547931 5.45702755 4.94186187 2.9311208 ]
[6.63777632 8.28396956 2.22856349 4.09215197]
[10.30482271  4.93193517  3.88564936  7.79688743]
[3.4351642  4.74684503 0.2259261  4.53043769]
[2.60663856 2.84742334 3.24149638 0.58788315]
[4.84554685 6.89132218 4.54685508 4.9600875 ]
[7.82849022 0.0681875  2.22044134 1.42418136]
[5.29536751 8.11960468 3.73088005 2.01884039]
[4.83762695 3.10545201 5.1328328  7.83125698]
[0.30728727 4.90426386 4.33455115 3.60816826]
[4.96071813 4.04671089 5.76946656 6.7154566 ]
[8.34517362 4.74012812 2.81596426 0.44480438]
[1.91566731 7.58986454 3.9742142  2.5942749 ]
[4.71791072 7.45447026 2.69547999 5.42725181]
[2.72884379 3.31291266 5.50094469 1.93693967]
[2.69717565 6.58967348 5.23742153 1.9165484 ]
[4.53533286 3.03925727 0.05449929 2.05135375]
[5.0296304  3.59639098 6.46566403 4.75225858]
[7.44537784 1.56497581 3.22206139 4.62951573]
[5.0976964  4.39120504 5.28929285 1.77930625]
[6.65857933 4.26235051 4.22618318 3.58842874]
[3.49170367 4.75373938 3.98453

KeyboardInterrupt: 